# F1 Data Scraping with FastF1

This notebook is used to scrape Formula 1 race results data using the FastF1 library and save it for further analysis.

In [ ]:
import fastf1
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

## Enable FastF1 Cache

Create a cache directory if it does not exist and enable FastF1's caching mechanism to speed up data retrieval.

In [ ]:
if not os.path.exists('fastf1_cache'):
    os.makedirs('fastf1_cache')

fastf1.Cache.enable_cache('fastf1_cache')

## Select Season and Get Race Schedule

Define the seasons of interest and select the current season. Retrieve the event schedule for the selected season.

In [ ]:
seasons = [2019, 2020, 2021, 2022, 2023, 2024]
curr_season = seasons[0]

In [ ]:
races = fastf1.get_event_schedule(curr_season, include_testing=False)

## Download Race Results for Each Event

Iterate through each race in the selected season, load the race session, and extract the results for each driver. Store all results in a list.

In [ ]:
races_df = pd.DataFrame(races[['EventName']]).set_index(races['RoundNumber'])

all_results = []

for rnd, row in races_df.iterrows():
    race = row['EventName']

    session = fastf1.get_session(curr_season, rnd, 'R')
    session.load()
    results = session.results
    results_df = pd.DataFrame(results[['DriverNumber', 'Abbreviation', 'DriverId', 'TeamId', 'Position']]).reset_index(drop=True)
    results_df['EventName'] = race
    results_df['Round'] = rnd
    results_df['Season'] = curr_season

    all_results.append(results_df)

final_results_df = pd.concat(all_results, ignore_index=True)

## Save Results to CSV

Save the combined race results for the selected season to a CSV file for future use.

In [ ]:
final_results_df.to_csv(f'data/final_results_{curr_season}.csv', index=False)